In [1]:
# Base python imports
import os
import sys
import time
import datetime

# Assuming the notebook is located in <ROOT>/notebooks
sys.path.append(os.path.abspath('..'))

#PyPi imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

# Custom imports
from src.modeling.utils.OpticalFlowUtils import FlowReader
from src.modeling.utils.OpticalFlowUtils import FlowVisualiser
from src.modeling.utils.MPISintelUtils import MPISintelHandler
from src.modeling.customML.customLosses.CustomLosses import EPE_Loss
from src.modeling.customML.customModels.OpticalFlowUnrolling import UnrolledOFModel

2024-08-04 17:50:05.645040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 17:50:06.448484: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 17:50:06.662766: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-04 17:50:08.263551: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 17:50:15.791852: W tensorflow/compiler/tf2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU(s): {gpus}")
if gpus:
    # Restrict TensorFlow to only allocate 8.1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=1024*8.0)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
        print(f"GPU(s): {gpus}")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(f"Runtime Error: {e}")

GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs
GPU(s): [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1722765029.716036  461414 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722765032.044226  461414 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722765032.044274  461414 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722765032.048305  461414 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722765032.048376  461414 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
reader = FlowReader()
visualiser = FlowVisualiser()
dataHandler = MPISintelHandler()

In [6]:
RAW_DATA_PATH = os.path.join("..", "data", "raw")
PROCESSED_DATA_PATH = os.path.join("..", "data", "processed")

IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "trainImages.data")
FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "trainFlows.data")

trainImgsArray = dataHandler.loadData(IMAGES_PATH)
trainFlowsArray = dataHandler.loadData(FLOWS_PATH)

print(f"Sintel images shape: {trainImgsArray.shape}")
print(f"Sintel flows shape: {trainFlowsArray.shape}")

Sintel images shape: (1041, 2, 436, 1024)
Sintel flows shape: (1041, 436, 1024, 2)


In [7]:
TRAIN_IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "trainX.data")
TRAIN_FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "trainy.data")

train_X = dataHandler.loadData(TRAIN_IMAGES_PATH)
train_X = train_X.astype('float32') / 255.0
train_X = tf.reshape(train_X, shape=(-1, 2, 436, 1024, 1))

train_y = dataHandler.loadData(TRAIN_FLOWS_PATH)
train_y = train_y.astype('float32')

TEST_IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "testX.data")
TEST_FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "testy.data")

test_X = dataHandler.loadData(TEST_IMAGES_PATH)
test_X = test_X.astype('float32') / 255.0
test_X = tf.reshape(test_X, shape=(-1, 2, 436, 1024, 1))

test_y = dataHandler.loadData(TEST_FLOWS_PATH)
test_y = test_y.astype('float32')

VAL_IMAGES_PATH = os.path.join(PROCESSED_DATA_PATH, "valX.data")
VAL_FLOWS_PATH = os.path.join(PROCESSED_DATA_PATH, "valy.data")

val_X = dataHandler.loadData(VAL_IMAGES_PATH)
val_X = val_X.astype('float32') / 255.0
val_X = tf.reshape(val_X, shape=(-1, 2, 436, 1024, 1))

val_y = dataHandler.loadData(VAL_FLOWS_PATH)
val_y = val_y.astype('float32')

2024-08-04 17:50:53.023569: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1821573120 exceeds 10% of free system memory.
2024-08-04 17:51:03.460609: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1060798464 exceeds 10% of free system memory.
2024-08-04 17:51:09.947576: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 835780608 exceeds 10% of free system memory.


In [8]:
print(f"train_X: {train_X.shape} test_X: {test_X.shape} val_X: {val_X.shape} -> Total Samples: {train_X.shape[0] + test_X.shape[0] + val_X.shape[0]}")
print(f"train_y: {train_y.shape} test_y: {test_y.shape} val_y: {val_y.shape} -> Total Samples: {train_y.shape[0] + test_y.shape[0] + val_y.shape[0]}")

train_X: (510, 2, 436, 1024, 1) test_X: (297, 2, 436, 1024, 1) val_X: (234, 2, 436, 1024, 1) -> Total Samples: 1041
train_y: (510, 436, 1024, 2) test_y: (297, 436, 1024, 2) val_y: (234, 436, 1024, 2) -> Total Samples: 1041


In [9]:
model = UnrolledOFModel(num_iterations = 15, name = "UnrollingOF")
model.summary()

Model: "UnrollingOF"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ DataTermLayer_1 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_2 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_3 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_4 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_5 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_6 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_7 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_8 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_9 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_10                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_11                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_12                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_13                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_14                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_15                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_1       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_2       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_3       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_4       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_5       │ ?                      │   0 (unbuilt

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
log_dir = os.path.join("..", "assets", "logs", "fits", f"{model.name}_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)

modelDir = os.path.join("..", "assets", "ml", "models", model.name)
if not os.path.exists(modelDir):
    os.makedirs(modelDir)

modelPath = os.path.join(modelDir, f"{model.name}_best.weights.h5")
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=modelPath, monitor='val_loss', save_best_only=True, save_weights_only=True)

learning_rate_callback = tf.keras.callbacks.ReduceLROnPlateau(factor=0.85, patience=50, min_lr=1e-6)
_callbacks = [tensorboard_callback, model_checkpoint_callback, learning_rate_callback]

callbacks = tf.keras.callbacks.CallbackList(_callbacks, add_history=False)

fileWriter = tf.summary.create_file_writer(os.path.join(log_dir, "metrics"))
fileWriter.set_as_default()

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

In [12]:
model.compile(optimizer=optimizer, loss=EPE_Loss(), metrics = ['mse'])
model.build(input_shape=(None, 2, 436, 1024, 1))
model.summary()

/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'UnrollingOF', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "UnrollingOF"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ DataTermLayer_1 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_2 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_3 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_4 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_5 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_6 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_7 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_8 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_9 (DataTermLayer) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_10                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_11                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_12                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_13                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_14                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ DataTermLayer_15                │ ?                      │   0 (unbuilt) │
│ (DataTermLayer)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_1       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_2       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_3       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_4       │ ?                      │   0 (unbuilt) │
│ (RegularisationTermLayer)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ RegularisationTermLayer_5       │ ?                      │   0 (unbuilt

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

: 

In [13]:
hist = model.fit(x=train_X, y=train_y, epochs = 2500, verbose = True, batch_size=1, validation_data=(test_X, test_y), callbacks=_callbacks)

2024-08-04 17:51:14.471533: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1821573120 exceeds 10% of free system memory.
2024-08-04 17:51:15.296905: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1821573120 exceeds 10% of free system memory.


Epoch 1/2500


/mnt/a/OneDrive/Repos/PIOFE-Unrolling/.venv-linux/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")
I0000 00:00:1722765080.392279  463639 service.cc:146] XLA service 0x7fd620039490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722765080.392331  463639 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2024-08-04 17:51:21.930762: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-04 17:51:23.109506: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1722765085.587105  463639 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the

510/510 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - loss: 114.9928 - mse: 47022.5703 - val_loss: 114.8224 - val_mse: 50637.6250 - learning_rate: 0.0010
Epoch 2/2500
510/510 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 111.2512 - mse: 46881.2070 - val_loss: 114.8224 - val_mse: 50637.6250 - learning_rate: 0.0010
Epoch 3/2500
510/510 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - loss: 112.3114 - mse: 47049.2148 - val_loss: 114.8224 - val_mse: 50637.6250 - learning_rate: 0.0010
Epoch 4/2500
510/510 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 111.0853 - mse: 44939.7148 - val_loss: 114.8224 - val_mse: 50637.6250 - learning_rate: 0.0010
Epoch 5/2500
510/510 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 110.1947 - mse: 44891.0039 - val_loss: 114.8224 - val_mse: 50637.6250 - learning_rate: 0.0010
Epoch 6/2500
510/510 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 111.8209 - mse: 46302.5820 - val_loss: 114.8224 - val_mse: 50637.6250 - learning_rate: 0.0010
Epoch 7/2500
510/510 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - loss: 114.628

In [ ]:
def visualiseFits(fits, validationData = True):
    fig, axs = plt.subplots(2, 2, figsize = (40, 20))

    axs[0][0].plot(fits.history['loss'], label = "Conventional NN Train EPE")
    axs[0][0].legend()
    axs[0][0].set_xlabel("Epochs")
    axs[0][0].set_ylabel("EPE (log Scaled)")
    axs[0][0].set_yscale('log')
    axs[0][0].grid(True)

    axs[1][0].plot(fits.history['loss'], label = "Conventional NN Train EPE")
    axs[1][0].legend()
    axs[1][0].set_xlabel("Epochs")
    axs[1][0].set_ylabel("EPE")
    axs[1][0].grid(True)

    axs[0][1].plot(fits.history['mse'], label = "Conventional NN Train MSE")
    axs[0][1].legend()
    axs[0][1].set_xlabel("Epochs")
    axs[0][1].set_ylabel("MSE")
    axs[0][1].grid(True)

    axs[1][1].plot(fits.history['mse'], label = "Conventional NN Train MSE")
    axs[1][1].legend()
    axs[1][1].set_xlabel("Epochs")
    axs[1][1].set_ylabel("MSE (log scaled)")
    axs[1][1].set_yscale('log')
    axs[1][1].grid(True)
    
    if validationData:
        axs[0][0].plot(fits.history['val_loss'], label = "Conventional NN Train EPE validation")
        axs[1][0].plot(fits.history['val_loss'], label = "Conventional NN Train EPE validation")
        axs[0][1].plot(fits.history['val_mse'], label = "Conventional NN Train MSE validation")
        axs[1][1].plot(fits.history['val_mse'], label = "Conventional NN Train MAE validation")

    plt.show()

In [ ]:
visualiseFits(hist)

In [ ]:
result = model.predict(train_X[0].numpy().reshape(1, 2, 436, 1024, 1))
result[0].shape

In [ ]:
visualiser.visualiseOpticalFlow(result[0])

In [ ]:
randomImagePair =tf.random.uniform(
    shape = (5, 2, 436, 1024, 1),
    minval=0,
    maxval=255,
    dtype=tf.dtypes.float32,
    seed=None,
    name=None
)

In [ ]:

fig, ax = plt.subplots(2, 1, figsize = (20, 10))

ax[0].imshow(randomImagePair[0, 0, :, :, :], cmap = 'gray')
ax[1].imshow(randomImagePair[0, 0, :, :, :], cmap = 'gray')
plt.show()

In [ ]:
result = model(randomImagePair)

In [ ]:
print(result[0])

In [ ]:
visualiser.visualiseOpticalFlow(flow=result[4].numpy())